# This project contains sensitive information. To protect the client's identity, I will not be reading the csv file that will allow the cells below to run. For more information, please refer to the README.md

This version of the program has everything in one cell, without any intricate explanations. There are very brief descriptions of what that chunk of code does, but nothing like the other notebook. 

In this notebook, we're making the whole process of cleaning and outputting into ONE function. This saves a lot of time, but is not practical. If you have an error, you have to know enough of how to read the error message in order to locate the error and fix it in this function. That is why I have the other notebook. With that, you can pinpoint any errors and decide if you want to output everything using the other notebook, or this one. 

All the way at the bottom, you'll see that we call the function three times. The only parameter that is needed is the filename, so exception_cleaner('/Users/filelocation/filename.csv') is enough to get everything done. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
def append_df_to_excel(filename, df, sheet_name = 'Sheet1', startrow = None, 
                       truncate_sheet = False, **to_excel_kwargs):
    from openpyxl import load_workbook
    
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')
        
    writer = pd.ExcelWriter(filename, engine = 'openpyxl')
    
    try:
        writer.book = load_workbook(filename)
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row
        
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            idx = writer.book.sheetnames.index(sheet_name)
            writer.book.remove(writer.book.worksheets[idx])
            writer.book.create_sheet(sheet_name, idx)
            
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
        
    except FileNotFoundError:
        pass
    
    if startrow is None:
        startrow = 0
        
    df.to_excel(writer, sheet_name, startrow = startrow, **to_excel_kwargs)
    
    writer.save()

In [3]:
def exception_cleaner(filename):
    VENDOR_LIST = pd.read_csv(filename, encoding='latin1')
    
    MONTH_ERROR = VENDOR_LIST[VENDOR_LIST['More than 6 months since status date?'].apply(lambda x: x == '#VALUE!')]

   
    VENDOR_LIST_FULL = VENDOR_LIST.replace(['None', 'nan', '', ' ', 'NONE', 'na', 'NA', 'unk', 'unknown', 'UNKNOWN', 'UNK', '#VALUE!'], np.nan, inplace = False)
    #this does c y z
    
    
    VENDOR_LIST_FULL['More than 6 months since status date?'] = VENDOR_LIST_FULL['More than 6 months since status date?'].astype(float) #makes object into float

    VALUE_COUNT = VENDOR_LIST['Living/Domicile Status'].value_counts()
    LIST_TYPE = VENDOR_LIST_FULL['List Type'].value_counts()
    
    #MONTH_ERROR = VENDOR_LIST[VENDOR_LIST['More than 6 months since status date?'] == '#VALUE!'] #string values in int column
    

    Missing_Demo = VENDOR_LIST_FULL[VENDOR_LIST_FULL.loc[:, :'Date Since'].isna().any(axis = 1)] #All cases with missing blue category
    INVALID_SOCIAL = VENDOR_LIST_FULL[VENDOR_LIST_FULL['S.S.#.'].str.len() < 11] #Social less than 9 digits
    NO_SOCIAL = VENDOR_LIST_FULL.loc[VENDOR_LIST_FULL['S.S.#.'].isnull()] #Missing Social
    
    LDS_DECEASED = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status']=='Deceased']
    #All Cases deceased

    LDS_DECEASED_SUB = LDS_DECEASED[LDS_DECEASED['Current Discharge Status'] == 'Submit D/C Request to HRA']
    #Cases Living / Domicile = Deceased AND Current Discharge Status. = Submit DC request to HRA
    
    LDS_NO_DISCHARGE = LDS_DECEASED[LDS_DECEASED.loc[: , 'Petition Date':"Vendor's Previous Comments"].isna().all(axis = 1)]
    #Client is deceased and nothing in columns Z-AE
    
    #Correct Criteria
    Current_Discharge_Status = ['Clt back in the community', 'Clt scheduled to return to the community within 6 months of admission',\
                            'Clt returned back to NYC', 'Successor guardian appointed FA to be submitted', 'Submit D/C Request to HRA',\
                            'Judge denied D/C', 'Successor guardian not yet commissioned', 'Vendor not seeking discharge', 'Clt discharged',\
                            'Pending Court Approval']
    
    Living_Domicile_Status = ['Deceased', 'Adult Home', 'Assisted Living', 'Supportive Housing', 'Hospital', 'Shelter',\
                          'Nursing Home-Perm', 'Nursing Home-Temp', 'Street Homeless', 'Incarcerated',\
                          'Resides in Community', 'Whereabouts unknown', 'Out of Jurisdiction', 'Substance abuse facility',\
                          'OPWDD facility', 'Congregate Care Facility']
    
    Institutionalized = ['Adult Home', 'Assisted Living', 'Supportive Housing', 'Hospital', 'Shelter',\
                          'Nursing Home-Perm', 'Nursing Home-Temp', 'Incarcerated','OPWDD facility', 'Congregate Care Facility']
    
    LDS_ERROR = VENDOR_LIST_FULL[~VENDOR_LIST_FULL['Living/Domicile Status'].isin(Living_Domicile_Status)] 
    #Cases with flawed Living/Domicile Status (NaN, typos, not in proper list)
    
    CDS_ERROR = VENDOR_LIST_FULL[~VENDOR_LIST_FULL['Current Discharge Status'].isin(Current_Discharge_Status)] 
    #Cases with flawed Current Discharge Status (I think they're all NaN. dropna() = 0 rows)
    
    #All Institutionalized
    L_D_S = ['Adult Home', 'Assisted Living', 'Supportive Housing', 'Hospital', 'Shelter', 'Nursing Home-Perm', 
             'Nursing Home-Temp', 'Incarcerated','OPWDD facility', 'Congregate Care Facility']
    
    LDS_removed = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'].isin(L_D_S)]
    #Living Domicile Status Without Deceased / Homeless / whereabouts / jurisdiction / resides in community / Substance Abuse
    
    LDS180 = LDS_removed[(LDS_removed['More than 6 months since status date?'] > 180) & (LDS_removed['More than 6 months since status date?'] < 365)]
    #cases institutionalized that have <365 and >180 status date
    LDS180_FOCUS = LDS180[LDS180.loc[: , 'Petition Date':"Vendor's Previous Comments"].isna().all(axis = 1)]
    #AND have nothing in columns Z-AE
    
    LDS365 = LDS_removed[LDS_removed['More than 6 months since status date?'] >= 365]
    #cases filtered that have >=365 status date
    LDS365_FOCUS = LDS365[LDS365.loc[: , 'Petition Date':"Vendor's Previous Comments"].isna().all(axis = 1)]
    #AND have nothing in coumns Z-AE
    
    #LDS without resides in community
    LDStatus = ['Deceased', 'Adult Home', 'Assisted Living', 'Supportive Housing', 'Hospital', 'Shelter',\
                          'Nursing Home-Perm', 'Nursing Home-Temp', 'Street Homeless', 'Incarcerated',\
                          'Whereabouts unknown', 'Out of Jurisdiction', 'Substance abuse facility',\
                          'OPWDD facility', 'Congregate Care Facility']
    
    LDStatusdf= VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'].isin(LDStatus)]

    LDSDateNull = LDStatusdf[LDStatusdf['Date Since'].isna()]
    #Cases where LDS != Residing in Community and Date Since is null
    
    LDStemp = LDStatusdf[LDStatusdf['Current Discharge Status'] == 'Vendor not seeking discharge']
    LDS_NoDIS_1YR = LDStemp[LDStemp['More than 6 months since status date?'] >= 365]
    #Anything other than resides in the community and date since is greater than 1 year and CDS = vendor not seeking discharge
    
    YPetition = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Petition Filed  '] == 'Yes']
    YPetitionNA = YPetition[YPetition['Petition Date'].isna()]
    #Yes to petition filed, yet has no petition date
    
    YPetitionUrgent = YPetition[YPetition.loc[: , 'Hearing Date' : 'Comments for Current Discharge Status'].isna().all(axis = 1)]
    #Yes Petition with nothing in columns AB-AE (Hearing Date to Comments for CDS)
    
    NPetition = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Petition Filed  '] == 'No']
    NPetitionVAL = NPetition[NPetition['Petition Date'].notna()]
    #No to petition filed, yet has a petition date
    
    CD = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Clt discharged']
    CD_Null_Date = CD[CD['D/C order Date'].isna()]
    #Clients discharged without discharge date
    
    Jurisdictiondf = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Out of Jurisdiction']
    JCDS_Null = Jurisdictiondf[Jurisdictiondf['Current Discharge Status'].isna()]
    #All Cases out of Jurisdiction that don't have a discharge status
    
    #HRA_APPROVAL_NULL = VENDOR_LIST_FULL[VENDOR_LIST_FULL["Days since HRA's Approval"].astype(float).isna()]
    
    NHT = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Nursing Home-Temp']
    NHT_HRA_NULL = NHT[NHT["Days since HRA's Approval"].isna()]
    NHT_DATE_180 = NHT_HRA_NULL[(NHT_HRA_NULL['More than 6 months since status date?'] >= 180) & (NHT_HRA_NULL['More than 6 months since status date?'] < 365)]
    #Nursing Home Temp Date Since is between 180 and 365 and days since HRA approval = NULL
    
    NHT_DATE_365 = NHT_HRA_NULL[NHT_HRA_NULL['More than 6 months since status date?'] >= 365]
    NHT_365_NO_PROGRESS = NHT_DATE_365[NHT_DATE_365.loc[:, 'Date Discharge Request Submitted to HRA':'Comments for Current Discharge Status'].isna().all(axis = 1)]
    #Nursing Home Temp Date Since is greater than 365 and days since HRA approval = NULL AND nothing in columns V-AE
    
    DENIED = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Judge denied D/C']
    DENIED_NA = DENIED[DENIED['Comments for Current Discharge Status'].isna()]
    #Judge Denied CDS and Comments are empty
        
    HRA_YES = VENDOR_LIST_FULL[VENDOR_LIST_FULL["HRA Approved (Y/N)"] == 'Yes']
    HRA = HRA_YES[HRA_YES.loc[:, 'Petition Filed  ':'D/C order Date'].isna().all(axis = 1)]
    HRA120 = HRA[(HRA["Days since HRA's Approval"] >= 60) & (HRA["Days since HRA's Approval"]<= 120)]
    #YES HRA Approval and days since 60 - 120 and nothing columns Z - AC
    HRA180 = HRA[(HRA["Days since HRA's Approval"] >= 120) & (HRA["Days since HRA's Approval"]<= 180)]
    #YES HRA Approval and days since 120 - 180 and nothing columns Z - AC
    HRA365 = HRA[(HRA["Days since HRA's Approval"] >= 180) & (HRA["Days since HRA's Approval"]<= 365)]
    #YES HRA Approval and days since 180 - 365 and nothing columns Z - AC
    HRA_SUBMIT_HRA = HRA[HRA['Current Discharge Status'] == 'Submit D/C Request to HRA']
    #YRS HRA Approval AND CDS = Submit DC Request to HRA
    
    LDS_WO_DEAD = ['Adult Home', 'Assisted Living', 'Supportive Housing', 'Hospital', 'Shelter',\
                          'Nursing Home-Perm', 'Nursing Home-Temp', 'Street Homeless', 'Incarcerated',\
                          'Resides in Community', 'Whereabouts unknown', 'Out of Jurisdiction', 'Substance abuse facility',\
                          'OPWDD facility', 'Congregate Care Facility']
    LDSWODEAD= VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'].isin(LDS_WO_DEAD)]
    HRA_DECEASED_NULL = LDSWODEAD[LDSWODEAD['HRA Approved (Y/N)'].isna()]
    #Cases where HRA Approval is NULL and the client is not deceased.
    
    
    #Respective CDS distributions
    CLT_COMMUNITY = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Clt back in the community']
    CLT_COMMUNITY_6MNTHS = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Clt scheduled to return to the community within 6 months of admission']
    DC_REQUEST = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Submit D/C Request to HRA']
    JUDGE_DENIED_DC = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Judge denied D/C']
    NO_SGUARDIAN = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Successor guardian not yet commissioned']
    VENDOR_NO_DISCHARGE = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Vendor not seeking discharge']
    CLT_RETURN_NYC = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Clt returned back to NYC']
    SGUARDIAN_FA = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Success guardian appointed FA to be submitted']
    CLT_DISCHARGED = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Clt discharged']
    PENDING_COURT_APP = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Pending Court Approval']
    
    #Respective LDS distributions
    DECEASED = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Deceased']
    ADULTHOME = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Adult Home']
    ASSISTED_LIVING = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Assisted Living']
    SUPPORTIVE_HOUSING = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Supporting Housing']
    HOSPITAL = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Hospital']
    SHELTER = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Shelter']
    NURSING_HOME_PERM = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Nursing Home-Perm']
    NURSING_HOME_TEMP = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Nursing Home-Temp']
    STREET_HOMELESS = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Street Homeless']
    INCARCERATED = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Incarcerated']
    RESIDES_COMMUNITY = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Resides in Community']
    WHEREABOUTS_UNK = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Whereabouts unknown']
    OUT_JURIS = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Out Of Jurisdiction']
    SUBSTANCE_ABUSE = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Substance abuse facility']
    OPWDD_FACILITY = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'OPWDD facility']
    CONGREGATE_CARE = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Congregate Care Facility']    
 

    #To be updated by Vendor
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', Missing_Demo, sheet_name = 'Missing_Demo')
    #All cases with missing blue category

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', INVALID_SOCIAL, sheet_name = 'Invalid_Social')
    #Social less than 9 digits

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NO_SOCIAL, sheet_name = 'No_Social')
    #Missing Social

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_DECEASED_SUB, sheet_name = 'Decd DC to HRA')
    #Cases Living / Domicile = Deceased AND Current Discharge Status. = Submit DC request to HRA

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_ERROR, sheet_name = 'LDS_ERROR')
    #Cases with flawed Living/Domicile Status (NaN, typos, not in proper list)

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CDS_ERROR, sheet_name = 'CDS_ERROR')
    #Cases with flawed Current Discharge Status (I think they're all NaN. dropna() = 0 rows)


    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDSDateNull, sheet_name = 'Date Since Required')
    #Cases where LDS != Residing in Community and Date Since is null
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', YPetitionNA, sheet_name = 'Y Petition N Date')
    #Yes to petition filed, yet has no petition date

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NPetitionVAL, sheet_name = 'N Petition Y Date')
    #No to petition filed, yet has a petition date

        
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', YPetitionUrgent, sheet_name = 'Y Petition No DC Info')
    #Yes Petition with nothing in columns AB-AE (Hearing Date to Comments for CDS)
        
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CD_Null_Date, sheet_name = 'CDS DC No DC Date')
    #Clients discharged without discharge date

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', JCDS_Null, sheet_name = 'Out of Juris No CDS')
    #All Cases out of Jurisdiction that don't have a discharge status
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS180_FOCUS, sheet_name = 'INS 180 to 365 No DC Info')
    #cases filtered that have 180-365 status date and have nothing in Z-AE

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS365_FOCUS, sheet_name = 'Ind 365 No DC Info')
    #cases filtered that have >=365 status date and have nothing in Z-AE

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HRA_SUBMIT_HRA, sheet_name = 'HRA App Y CDC Submit HRA')
    #YRS HRA Approval AND CDS = Submit DC Request to HRA

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', DENIED_NA, sheet_name = 'CDS JDG and No Comment')
    #Judge Denied CDS and Comments are empty
    


#INTERNAL REVIEW NEEDED
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', VALUE_COUNT, sheet_name = 'Value_Count')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LIST_TYPE, sheet_name = 'List Type Count')
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', MONTH_ERROR, sheet_name = 'Month_Error')
    #Cases that have #VALUE! in Month column
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_DECEASED, sheet_name = 'LDS_DECEASED')
    #Cases where LDS == Deceased

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_removed, sheet_name = 'Inst Clts')
    #Living Domicile Status Without Deceased / Homeless / whereabouts / jurisdiction / resides in community

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS180, sheet_name = 'Inst more than 180 days')
    #cases filtered that have >180 status date

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS365, sheet_name = 'Inst more than 365 days')
    #cases filtered that have >365 status date
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NHT_DATE_180, sheet_name = 'NHT more 180 No HRA Appr')
    #Nursing Home Temp Date Since is between 180 and 365 and days since HRA approval = NULL
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NHT_365_NO_PROGRESS, sheet_name = 'NHT more 365 No HRA DC Info')
    #Nursing Home Temp Date Since is greater than 365 and days since HRA approval = NULL AND nothing in columns V-AE
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_NO_DISCHARGE, sheet_name = 'LDS Deceased No DC Info')
    #Client is deceased and nothing in columns Z-AE
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_NoDIS_1YR, sheet_name = 'Date since more 1yr CDC VNSD')
    #Anything other than resides in the community and date since is greater than 1 year and CDS = vendor not seeking discharge

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HRA120, sheet_name = 'HRA Y more 60 No DC Info')
    #YES HRA Approval and days since 60 - 120 and nothing columns Z - AC
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HRA180, sheet_name = 'HRA Y more 180 No DC Info')
    #YES HRA Approval and days since 120 - 180 and nothing columns Z - AC
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HRA365, sheet_name = 'HRA Y more 365 No DC Info')
    #YES HRA Approval and days since 180 - 365 and nothing columns Z - AC
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HRA_DECEASED_NULL, sheet_name = 'HRA_DECEASED_BLANK')
    #Cases where HRA Approval is NULL and the client is not deceased.
    

    #LDS
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', DECEASED, sheet_name = 'DECEASED')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', ADULTHOME, sheet_name = 'ADULTHOME')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', ASSISTED_LIVING, sheet_name = 'ASSISTED_LIVING')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', SUPPORTIVE_HOUSING, sheet_name = 'SUPPORTIVE_HOUSING')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HOSPITAL, sheet_name = 'HOSPITAL')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', SHELTER, sheet_name = 'SHELTER')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NURSING_HOME_PERM, sheet_name = 'NURSING_HOME_PERM')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NURSING_HOME_TEMP, sheet_name = 'NURSING_HOME_TEMP')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', STREET_HOMELESS, sheet_name = 'STREET_HOMELESS')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', INCARCERATED, sheet_name = 'INCARCERATED')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', RESIDES_COMMUNITY, sheet_name = 'RESIDES_COMMUNITY')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', WHEREABOUTS_UNK, sheet_name = 'WHEREABOUTS_UNK')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', OUT_JURIS, sheet_name = 'OUT_JURIS')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', SUBSTANCE_ABUSE, sheet_name = 'SUBSTANCE_ABUSE')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', OPWDD_FACILITY, sheet_name = 'OPWDD_FACILITY')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CONGREGATE_CARE, sheet_name = 'CONGREGATE_CARE')

    #CDS
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CLT_COMMUNITY, sheet_name = 'CLT_COMMUNITY')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CLT_COMMUNITY_6MNTHS, sheet_name = 'CLT_COMMUNITY_6MNTHS')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', DC_REQUEST, sheet_name = 'DC_REQUEST')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', JUDGE_DENIED_DC, sheet_name = 'JUDGE_DENIED_DC')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NO_SGUARDIAN, sheet_name = 'NO_SGUARDIAN')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', VENDOR_NO_DISCHARGE, sheet_name = 'VENDOR_NO_DISCHARGE')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CLT_RETURN_NYC, sheet_name = 'CLT_RETURN_NYC')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', SGUARDIAN_FA, sheet_name = 'SGUARDIAN_FA')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CLT_DISCHARGED, sheet_name = 'CLT_DISCHARGED')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', PENDING_COURT_APP, sheet_name = 'PENDING_COURT_APP')


In [85]:
exception_cleaner('/Users/filelocation/filename1.csv')

In [82]:
exception_cleaner('/Users/filelocation/filename2.csv')

In [81]:
exception_cleaner('/Users/filelocation/filename3.csv')